1. Which geographic area of interest (AOI) are you focussing on?  This can be the same AOI you selected for written assignment 1, or you can focus on a new AOI. 

2. Describe your AOI in terms of it’s landscape characteristics: what are the dominant LULC categories?  Strictly speaking: do these qualify as purely land cover, land use, or combinations of land cover and land use?  

3. Does the forest in your AOI have what you believe to be spectrally and ecologically distinct sub-types?  If so, what are they? And what spectral and / or ecological characteristics allow differentiation? 

4. How accessible is information representing LULC for your area of interest?  What about data relating to socio-cultural and socio-economic attributes, governance, and land-use rights?  And biodiversity?  

5. Who are the primary stakeholders associated with the forests in your AOI? 

6. How would you describe the biodiversity associated with the forests in your AOI? 

7. Does carbon stock data exist specifically for the forests in your AOI (i.e., based on the results of a relevant forest inventory)?  If yes, did this inventory include forest stratification?  Which inventory methodology was used? And which carbon pools were assessed?  What about non-forest carbon stock values (e.g., deforested areas)? 

8. Are there allometric equations that were developed based on the forests in your AOI?  If yes, who developed them? If not, what are the closest/most appropriate allometric equations for you AOI? (find a regional, national, or international source)

9. Is there growth and yield data available that’s applicable to the forests in your AOI?  If yes, what is the source? 

In [1]:
import ee
import geopandas as gpd
import geemap

In [2]:
ee.Authenticate()
ee.Initialize()

In [ ]:
legends = geemap.builtin_legends
for legend in legends:
    print(legend)

In [ ]:
# In case of pyproj issues, specify proj data directory manually
# pyproj.datadir.set_data_dir('/opt/homebrew/bin/proj/')

In [ ]:
project_gdf = gpd.read_file(
    "../geometries/WDPA_WDOECM_Nov2024_Public_26621_shp/WDPA_WDOECM_Nov2024_Public_26621_shp-polygons.shp"
)
project_fc = ee.FeatureCollection(project_gdf.__geo_interface__)

## Copernicus Global Land Cover Layers: CGLS-LC100 

Copernicus GLC layers are available at 100m resolution for the period 2015-2019.
GCL images provide a discrete classification of a pixel, a quality indicator of the discrete classifsication, and a percent coverfraction of crops, grass, moss, shrubs, trees, and others. The forest type (5 classes) are also provided.

This source is interesting for it's medium resolution and multiple forest type classifications, which are desirable for biomass estimates in this area.

In [49]:
land_cover_collection = ee.Image(
    "COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019"
).select("discrete_classification")
land_cover_collection

In [48]:
forest_type = ee.Image("COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019").select(
    "forest_type"
)
forest_type

In [50]:
cgls_clip = land_cover_collection.clip(project_fc)
forest_type_clip = forest_type.clip(project_fc)

In [61]:
legend_dict = {
    # "0 Unknown": "282828",
    "20 Shrubs": "ffbb22",
    "30 Herbaceous Vegetation": "ffff4c",
    "40 Cultivated and Managed Vegetation/Agriculture": "f096ff",
    "50 Urban/Built Up": "fa0000",
    "60 Bare/Sparse Vegetation": "b4b4b4",
    # "70 Snow and Ice": "f0f0f0",
    "80 Permanent Water Bodies": "0032c8",
    "90 Herbaceous Wetland": "0096a0",
    # "100 Moss and Lichen": "fae6a0",
    "111 Closed Forest, Evergreen Needle Leaf": "58481f",
    "112 Closed Forest, Evergreen Broad Leaf": "009900",
    "113 Closed Forest, Deciduous Needle Leaf": "70663e",
    "114 Closed Forest, Deciduous Broad Leaf": "00cc00",
    "115 Closed Forest, Mixed": "4e751f",
    "116 Closed Forest, Other": "007800",
    "121 Open Forest, Evergreen Needle Leaf": "666000",
    "122 Open Forest, Evergreen Broad Leaf": "8db400",
    "123 Open Forest, Deciduous Needle Leaf": "8d7400",
    "124 Open Forest, Deciduous Broad Leaf": "a0dc00",
    "125 Open Forest, Mixed": "929900",
    "126 Open Forest, Other": "648c00",
    # "200 Oceans/Seas": "000080"
}

In [62]:
p = geemap.Map()
p.centerObject(project_fc)
p.addLayer(cgls_clip, {}, "Copernicus Global Land Cover Layers")
text = "Made by Ilse Paniagua"
p.add_text(text)
p.add_legend(title="", legend_dict=legend_dict)
p

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [74]:
def calculate_land_cover_area(image, region, scale=100):
    # Create a dictionary to hold the area results
    class_areas = image.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=region,
        scale=scale,
        maxPixels=1e9,
    )
    return class_areas

In [76]:
# Get area per land cover category in the region
class_areas = calculate_land_cover_area(cgls_clip, project_fc)

# Convert to a dictionary and calculate the percentage
area_dict = class_areas.get("discrete_classification").getInfo()
total_pixels = sum(area_dict.values())
percentage_dict = {
    key: (value / total_pixels) * 100 for key, value in area_dict.items()
}

In [78]:
percentage_dict

{'112': 60.37985138255459,
 '116': 0.6004483307490583,
 '122': 1.6499033209097502,
 '126': 7.1444635834924854,
 '20': 0.36259723975047076,
 '30': 1.0725220738492236,
 '40': 27.201682400989885,
 '50': 0.07922441152390942,
 '80': 0.6466605200492257,
 '90': 0.8626467361314002}

In [83]:
cgls_shortened_dict = {
    20: "Shrubs",
    30: "Herbaceous Vegetation",
    40: "Cultivated and Managed Vegetation/Agriculture",
    50: "Urban/Built Up",
    60: "Bare/Sparse Vegetation",
    80: "Permanent Water Bodies",
    90: "Herbaceous Wetland",
    111: "Closed Forest, Evergreen Needle Leaf",
    112: "Closed Forest, Evergreen Broad Leaf",
    113: "Closed Forest, Deciduous Needle Leaf",
    114: "Closed Forest, Deciduous Broad Leaf",
    115: "Closed Forest, Mixed",
    116: "Closed Forest, Other",
    121: "Open Forest, Evergreen Needle Leaf",
    122: "Open Forest, Evergreen Broad Leaf",
    123: "Open Forest, Deciduous Needle Leaf",
    124: "Open Forest, Deciduous Broad Leaf",
    125: "Open Forest, Mixed",
    126: "Open Forest, Other",
}


In [89]:
import pandas as pd

land_cover_df = pd.DataFrame.from_dict(percentage_dict, orient="index").reset_index()
land_cover_df.columns = ["land_value", "percentage"]
land_cover_df

,land_value,percentage
0,112,60.379851
1,116,0.600448
2,122,1.649903
3,126,7.144464
4,20,0.362597
5,30,1.072522
6,40,27.201682
7,50,0.079224
8,80,0.646661
9,90,0.862647


In [92]:
land_cover_df["land_cover"] = land_cover_df["land_value"].apply(
    lambda x: cgls_shortened_dict.get(int(x), "Unknown")
)

In [100]:
land_cover_df.sort_values("percentage", ascending=False, inplace=True)
land_cover_df["percentage"] = land_cover_df["percentage"].apply(lambda x: round(x, 2))
print(land_cover_df.to_markdown())

|    |   land_value |   percentage | land_cover                                    |
|---:|-------------:|-------------:|:----------------------------------------------|
|  0 |          112 |        60.38 | Closed Forest, Evergreen Broad Leaf           |
|  6 |           40 |        27.2  | Cultivated and Managed Vegetation/Agriculture |
|  3 |          126 |         7.14 | Open Forest, Other                            |
|  2 |          122 |         1.65 | Open Forest, Evergreen Broad Leaf             |
|  5 |           30 |         1.07 | Herbaceous Vegetation                         |
|  9 |           90 |         0.86 | Herbaceous Wetland                            |
|  8 |           80 |         0.65 | Permanent Water Bodies                        |
|  1 |          116 |         0.6  | Closed Forest, Other                          |
|  4 |           20 |         0.36 | Shrubs                                        |
|  7 |           50 |         0.08 | Urban/Built Up              

In [71]:
legend_dict = {
    "0 Unknown": "282828",
    "1 Evergreen needle leaf": "666000",
    "2 Evergreen broad leaf": "009900",
    "3 Deciduous needle leaf": "70663e",
    "4 Deciduous broad leaf": "a0dc00",
    "5 Mix of forest types": "929900",
}

In [72]:
m = geemap.Map()
m.centerObject(project_fc)
m.addLayer(forest_type_clip, {}, "Forest Type")
text = "Made by Ilse Paniagua"
m.add_text(text)
m.add_legend(legend_dict=legend_dict, title="Forest Types")
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [42]:
roi = p.user_roi
file_name = "../maps/cgls_mayan_biosphere_classes.png"

In [47]:
p.to_image(filename=file_name, monitor=0)


## ESA

In [28]:
esa_collection = ee.ImageCollection("ESA/WorldCover/v100").first()
esa_collection


In [ ]:
esa_collection.clip(project_fc)

In [27]:
visualization = {
    "bands": ["Map"],
}

m = geemap.Map()
m.center_object(dataset)
m.add_layer(dataset, visualization, "Landcover")
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…